In [ ]:
### Analize and import wifi dataset

In [25]:
import os
import csv
import json

import numpy as np
import pandas as pd

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [27]:
df=pd.read_csv("wardrive_elasticsearch-01.kismet.csv", delimiter=";")

In [28]:
df.head()

,Network,NetType,ESSID,BSSID,Info,Channel,Cloaked,Encryption,Decrypted,MaxRate,...,GPSMaxLon,GPSMaxAlt,GPSMaxSpd,GPSBestLat,GPSBestLon,GPSBestAlt,DataSize,IPType,IP,Unnamed: 38
0,1,infrastructure,Patricia,C8:D7:19:5D:26:25,NaN,10,No,"WPA2,WPA,AES-CCM,TKIP",No,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
1,2,infrastructure,bzn,4C:F9:5D:A6:25:B4,NaN,6,No,"WPA2,AES-CCM",No,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
2,3,infrastructure,ASUSR,2C:56:DC:F9:E4:60,NaN,1,No,"WPA2,AES-CCM",No,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
3,4,infrastructure,Netis 14 Spalatorie,04:8D:38:7E:C4:37,NaN,9,No,"WPA2,AES-CCM,TKIP",No,270.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
4,5,infrastructure,edimax,80:1F:02:E6:00:1F,NaN,2,No,"WPA2,AES-CCM",No,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN


In [29]:
df.shape

(5248, 39)

In [30]:
df.columns

Index(['Network', 'NetType', 'ESSID', 'BSSID', 'Info', 'Channel', 'Cloaked',
       'Encryption', 'Decrypted', 'MaxRate', 'MaxSeenRate', 'Beacon', 'LLC',
       'Data', 'Crypt', 'Weak', 'Total', 'Carrier', 'Encoding', 'FirstTime',
       'LastTime', 'BestQuality', 'BestSignal', 'BestNoise', 'GPSMinLat',
       'GPSMinLon', 'GPSMinAlt', 'GPSMinSpd', 'GPSMaxLat', 'GPSMaxLon',
       'GPSMaxAlt', 'GPSMaxSpd', 'GPSBestLat', 'GPSBestLon', 'GPSBestAlt',
       'DataSize', 'IPType', 'IP', 'Unnamed: 38'],
      dtype='object')

In [31]:
df_short=df[['Network', 'NetType', 'ESSID', 'BSSID','Channel','Encryption','MaxRate']]

In [32]:
df_short.isnull().sum()

Network         0
NetType         0
ESSID         173
BSSID           0
Channel         0
Encryption      0
MaxRate         0
dtype: int64

In [33]:
df_short=df_short.dropna()

In [34]:
df_short.isnull().sum()

Network       0
NetType       0
ESSID         0
BSSID         0
Channel       0
Encryption    0
MaxRate       0
dtype: int64

In [37]:
df_short.head()

,Network,NetType,ESSID,BSSID,Channel,Encryption,MaxRate
0,1,infrastructure,Patricia,C8:D7:19:5D:26:25,10,"WPA2,WPA,AES-CCM,TKIP",130.0
1,2,infrastructure,bzn,4C:F9:5D:A6:25:B4,6,"WPA2,AES-CCM",130.0
2,3,infrastructure,ASUSR,2C:56:DC:F9:E4:60,1,"WPA2,AES-CCM",130.0
3,4,infrastructure,Netis 14 Spalatorie,04:8D:38:7E:C4:37,9,"WPA2,AES-CCM,TKIP",270.0
4,5,infrastructure,edimax,80:1F:02:E6:00:1F,2,"WPA2,AES-CCM",135.0


In [39]:
df_short['Encryption'].value_counts()

WPA2,AES-CCM             2005
WPA2,AES-CCM,TKIP        1358
WPA2,WPA,AES-CCM          799
WPA2,WPA,AES-CCM,TKIP     557
OPN,None                  203
WPA,AES-CCM                72
WEP                        27
WPA,TKIP                   21
WPA2,TKIP                  20
WPA,AES-CCM,TKIP            8
WPA2,WPA,TKIP               2
WPA2,OPN,AES-CCM,TKIP       2
WPA2,None                   1
Name: Encryption, dtype: int64

In [38]:
es = Elasticsearch(["127.0.0.1:9200"])

es.indices.delete(index="wardrive",ignore=404)
docs = df_short.to_dict(orient='records')
bulk(es, docs, index='wardrive',doc_type='wifi', raise_on_error=True)
es.indices.refresh()

{'_shards': {'total': 11, 'successful': 6, 'failed': 0}}